In [ ]:
import wave
from balacoon_tts import TTS
import time
import threading
from TTS.api import TTS

: 

In [110]:
!export PYTHONPATH="/nlp/data/awadhawan/s2st/translatotron2/tts"
!echo $PYTHONPATH

In [91]:
# Balacoon TTS
tts = TTS("models/en_us_hifi92_e2ept.addon")
supported_speakers = tts.get_speakers()
speaker = supported_speakers[-1] if supported_speakers else ""
print(supported_speakers)


In [116]:
#Coqui TTS
tts = TTS(model_name="tts_models/multilingual/multi-dataset/your_tts", progress_bar=False, gpu=True)


TypeError: 'module' object is not callable

In [92]:
text ="A muffin is an individually portioned baked product, however the term can refer to one of two distinct items: a part-raised flatbread (like a crumpet) that is baked and then cooked on a griddle (typically unsweetened), or a (often sweetened) quickbread that is chemically leavened and then baked in a mold."

In [93]:
def streaming_write(sentence):
    for word in sentence.split():
        with open("streaming_input.txt", "a") as file:
            file.write(word+" ")
        time.sleep(0.2)

In [95]:
write_thread = threading.Thread(target=streaming_write, args = (text,) )
write_thread.start()

In [96]:
def convert_to_audio(chunk, name, speaker):
    # samples = tts.synthesize(chunk, speaker)
    # with wave.open("outputs/example"+ str(name) +".wav", "w") as fp:
    #     fp.setparams((1, 2, 24000, len(samples), "NONE", "NONE"))
    #     fp.writeframes(samples)
    tts.tts_to_file(chunk, speaker_wav="my/cloning/audio.wav", language="en", file_path="outputs/example"+ str(name) +".wav")

        
def streamingTTS():
    content=""
    while True:
        with open("streaming_input.txt", "r") as file:
            new_content = file.read()
            print(len(content), len(new_content))
            if len(new_content) > len(content):
                convert_to_audio(new_content[len(content):], len(new_content), speaker)
                content = new_content
            else:
                break
        time.sleep(1.2)


In [97]:
tts_thread = threading.Thread(target=streamingTTS)
tts_thread.start()

0 15


15 65
65 90
90 134
134 165
165 194
194 242
242 285
285 307
307 307
